# Pedestrian Crash - Trying to predict XYZ at the time of impact
## Read the dataset

Edo tha prospathiso na kano regress to XYZ ti stigmi tou Crash.

## Importing the libraries

In [2]:
from pathlib import Path
import sys
sys.path.insert(0, "C:\\Users\geork\projects\AIThesis\src\src")
import matplotlib.pyplot as plt

from dataset_reader import DatasetReaderCSV

## Read the dataset

In [28]:
path = Path("C:\\Users\geork\projects\AIThesis\src\\analysis\\3\datasets\my_dataset_all.csv")
reader = DatasetReaderCSV(path)
reader.read()
df = reader.convert_to_dataframe()
df = df.drop(columns=["Id", "Position", "Path", "GifPath", "Head_Collision_XYZ_index", "CarProfile"], errors="ignore")

In [29]:
print(df.dtypes)

Velocity               int64
Translation            int64
Rotation               int64
Head_Collision_X     float64
Head_Collision_Y     float64
Head_Collision_Z     float64
Front_Height           int64
Hood_Front_Width       int64
Hood_Back_Width        int64
Hood_Length            int64
Hood_Angle           float64
Windscreen_Length      int64
Windscreen_Angle       int64
dtype: object


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming df is your DataFrame
# Define features (X) and targets (y)
X = df.drop(columns=['Head_Collision_X', 'Head_Collision_Y', 'Head_Collision_Z'])
y = df[['Head_Collision_X', 'Head_Collision_Y', 'Head_Collision_Z']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(3)  # 3 output neurons for Head_Collision_X, Head_Collision_Y, Head_Collision_Z
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, batch_size=32)

c:\Users\geork\projects\AIThesis\src\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 485449.2188 - val_loss: 472014.3438
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 463398.5938 - val_loss: 459963.1250
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 454468.7812 - val_loss: 411237.7812
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 385705.3438 - val_loss: 284349.2500
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 242821.3281 - val_loss: 123165.7188
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 104930.2266 - val_loss: 73823.0547
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 68081.5781 - val_loss: 65183.9492
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 62334.0703 - val_loss: 61540.1914
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 54686.0898 - val_loss: 58770.2422
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 54248.2070 - val_loss: 56521.0625
Epoch 11/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/st

In [32]:
# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Make predictions
y_pred = model.predict(X_test)

# If you want to see the predictions
predictions_df = pd.DataFrame(y_pred, columns=['Head_Collision_X', 'Head_Collision_Y', 'Head_Collision_Z'])
print(predictions_df.head())

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 20860.4277  
Test Loss: 20750.23046875
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step
   Head_Collision_X  Head_Collision_Y  Head_Collision_Z
0        335.588409         28.527544       1099.462402
1        540.538879       -158.257431       1262.713257
2        566.672424        445.154877       1253.069702
3        252.260147         14.364577        983.750000
4        239.724258        171.387436       1174.449219
